In [1191]:
#pandas: CSV形式の表を操作するためのライブラリ
#numpy: 数値の配列を効率よく計算するためのライブラリ
#matplotlib: グラフ描画ライブラリ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import codecs
from sklearn import preprocessing

import seaborn as sns
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [1192]:
#マジックコメント
%matplotlib inline

In [1193]:
#DataFrameで全ての列を表示するよう設定
pd.options.display.max_columns = None

In [1194]:
#データの読み込み，データ型の確認
with codecs.open("AllTokyo_202001_202109_kodate.csv", "r", "Shift-JIS", "ignore") as dataset:
    df = pd.read_table(dataset, delimiter=",")

In [1195]:
#最初の5行を表示する
df.head()

,No,種類,地域,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,取引価格（総額）,面積（u）,土地の形状,間口,延床面積（u）,建築年,建物の構造,用途,今後の利用目的,前面道路：方位,前面道路：種類,前面道路：幅員（ｍ）,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等
0,163,宅地(土地と建物),商業地,13101,東京都,千代田区,神田佐久間町,秋葉原,4,260000000,85,ほぼ長方形,7,470,昭和44年,ＲＣ,事務所、作業場、倉庫、駐車場,住宅,北,区道,16.0,商業地域,80.0,600.0,2020年第４四半期,NaN,NaN
1,169,宅地(土地と建物),住宅地,13101,東京都,千代田区,神田佐久間町,秋葉原,4,85000000,115,長方形,8.2,30,昭和37年,木造,倉庫,住宅,北,私道,2.2,商業地域,80.0,500.0,2020年第１四半期,NaN,NaN
2,238,宅地(土地と建物),商業地,13101,東京都,千代田区,神田須田町,神田(東京),2,230000000,70,ほぼ長方形,7.6,230,昭和60年,鉄骨造,事務所,住宅,北西,区道,6.0,商業地域,80.0,600.0,2020年第２四半期,NaN,NaN
3,239,宅地(土地と建物),商業地,13101,東京都,千代田区,神田須田町,神田(東京),1,240000000,70,ほぼ長方形,7.6,230,昭和47年,鉄骨造,住宅、事務所、駐車場,住宅,北西,区道,6.0,商業地域,80.0,600.0,2020年第２四半期,NaN,NaN
4,240,宅地(土地と建物),商業地,13101,東京都,千代田区,神田須田町,神田(東京),2,140000000,40,ほぼ長方形,6,65,NaN,NaN,NaN,住宅,南東,区道,3.0,商業地域,80.0,600.0,2020年第２四半期,NaN,NaN


In [1196]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16787 entries, 0 to 16786
Data columns (total 27 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   No          16787 non-null  int64  
 1   種類          16787 non-null  object 
 2   地域          16787 non-null  object 
 3   市区町村コード     16787 non-null  int64  
 4   都道府県名       16787 non-null  object 
 5   市区町村名       16787 non-null  object 
 6   地区名         16787 non-null  object 
 7   最寄駅：名称      16768 non-null  object 
 8   最寄駅：距離（分）   16764 non-null  object 
 9   取引価格（総額）    16787 non-null  int64  
 10  面積（u）       16787 non-null  object 
 11  土地の形状       16782 non-null  object 
 12  間口          15264 non-null  object 
 13  延床面積（u）     16285 non-null  object 
 14  建築年         16093 non-null  object 
 15  建物の構造       15970 non-null  object 
 16  用途          15965 non-null  object 
 17  今後の利用目的     16787 non-null  object 
 18  前面道路：方位     16787 non-null  object 
 19  前面道路：種類     16676 non-nul

In [1197]:
df.isnull().sum()

No                0
種類                0
地域                0
市区町村コード           0
都道府県名             0
市区町村名             0
地区名               0
最寄駅：名称           19
最寄駅：距離（分）        23
取引価格（総額）          0
面積（u）             0
土地の形状             5
間口             1523
延床面積（u）         502
建築年             694
建物の構造           817
用途              822
今後の利用目的           0
前面道路：方位           0
前面道路：種類         111
前面道路：幅員（ｍ）      273
都市計画             65
建ぺい率（％）         103
容積率（％）          103
取引時点              0
改装            16787
取引の事情等        14675
dtype: int64

In [1198]:
#データのうち，Noと欠損値の数が10000以上の列を削除する．
df=df.drop('改装', axis=1)
df=df.drop('取引の事情等', axis=1)
df=df.drop('No', axis=1)

#最寄駅名は，市区町村に依存してしまう&ダミー変数化したときに処理が重くなる&欠損値が19個あるので削除．市区町村コードも不要なので削除．
df=df.drop('最寄駅：名称', axis=1)
df=df.drop('市区町村コード', axis=1)
#東京都のデータしかないので，都道府県名を削除．
df=df.drop('都道府県名', axis=1)

#接面道路の欠損値は，接面道路がなかったことにする．
df=df.fillna({'前面道路：種類':'接面道路無し故に種類無し'})
df.isnull().sum()

種類               0
地域               0
市区町村名            0
地区名              0
最寄駅：距離（分）       23
取引価格（総額）         0
面積（u）            0
土地の形状            5
間口            1523
延床面積（u）        502
建築年            694
建物の構造          817
用途             822
今後の利用目的          0
前面道路：方位          0
前面道路：種類          0
前面道路：幅員（ｍ）     273
都市計画            65
建ぺい率（％）        103
容積率（％）         103
取引時点             0
dtype: int64

In [1199]:
df.head()

,種類,地域,市区町村名,地区名,最寄駅：距離（分）,取引価格（総額）,面積（u）,土地の形状,間口,延床面積（u）,建築年,建物の構造,用途,今後の利用目的,前面道路：方位,前面道路：種類,前面道路：幅員（ｍ）,都市計画,建ぺい率（％）,容積率（％）,取引時点
0,宅地(土地と建物),商業地,千代田区,神田佐久間町,4,260000000,85,ほぼ長方形,7,470,昭和44年,ＲＣ,事務所、作業場、倉庫、駐車場,住宅,北,区道,16.0,商業地域,80.0,600.0,2020年第４四半期
1,宅地(土地と建物),住宅地,千代田区,神田佐久間町,4,85000000,115,長方形,8.2,30,昭和37年,木造,倉庫,住宅,北,私道,2.2,商業地域,80.0,500.0,2020年第１四半期
2,宅地(土地と建物),商業地,千代田区,神田須田町,2,230000000,70,ほぼ長方形,7.6,230,昭和60年,鉄骨造,事務所,住宅,北西,区道,6.0,商業地域,80.0,600.0,2020年第２四半期
3,宅地(土地と建物),商業地,千代田区,神田須田町,1,240000000,70,ほぼ長方形,7.6,230,昭和47年,鉄骨造,住宅、事務所、駐車場,住宅,北西,区道,6.0,商業地域,80.0,600.0,2020年第２四半期
4,宅地(土地と建物),商業地,千代田区,神田須田町,2,140000000,40,ほぼ長方形,6,65,NaN,NaN,NaN,住宅,南東,区道,3.0,商業地域,80.0,600.0,2020年第２四半期


In [1200]:
#前面道路の種類ごとに，幅員の平均値を算出して欠損値に代入する．

df.loc[(df["前面道路：種類"].values == "区道") & (df["前面道路：幅員（ｍ）"].isnull()), "前面道路：幅員（ｍ）"] = df[df["前面道路：種類"] == "区道"]["前面道路：幅員（ｍ）"].mean()
df.loc[(df["前面道路：種類"].values == "市道") & (df["前面道路：幅員（ｍ）"].isnull()), "前面道路：幅員（ｍ）"] = df[df["前面道路：種類"] == "市道"]["前面道路：幅員（ｍ）"].mean()
df.loc[(df["前面道路：種類"].values == "私道") & (df["前面道路：幅員（ｍ）"].isnull()), "前面道路：幅員（ｍ）"] = df[df["前面道路：種類"] == "私道"]["前面道路：幅員（ｍ）"].mean()
df.loc[(df["前面道路：種類"].values == "町道") & (df["前面道路：幅員（ｍ）"].isnull()), "前面道路：幅員（ｍ）"] = df[df["前面道路：種類"] == "町道"]["前面道路：幅員（ｍ）"].mean()
df.loc[(df["前面道路：種類"].values == "都道") & (df["前面道路：幅員（ｍ）"].isnull()), "前面道路：幅員（ｍ）"] = df[df["前面道路：種類"] == "都道"]["前面道路：幅員（ｍ）"].mean()
df.loc[(df["前面道路：種類"].values == "道道") & (df["前面道路：幅員（ｍ）"].isnull()), "前面道路：幅員（ｍ）"] = df[df["前面道路：種類"] == "道道"]["前面道路：幅員（ｍ）"].mean()
df.loc[(df["前面道路：種類"].values == "道路") & (df["前面道路：幅員（ｍ）"].isnull()), "前面道路：幅員（ｍ）"] = df[df["前面道路：種類"] == "道路"]["前面道路：幅員（ｍ）"].mean()
df.isnull().sum()

種類               0
地域               0
市区町村名            0
地区名              0
最寄駅：距離（分）       23
取引価格（総額）         0
面積（u）            0
土地の形状            5
間口            1523
延床面積（u）        502
建築年            694
建物の構造          817
用途             822
今後の利用目的          0
前面道路：方位          0
前面道路：種類          0
前面道路：幅員（ｍ）     111
都市計画            65
建ぺい率（％）        103
容積率（％）         103
取引時点             0
dtype: int64

In [1201]:
#道路に接していない案件の幅員を0[m]とする．
df=df.fillna({'前面道路：幅員（ｍ）':0})
df.isnull().sum()

種類               0
地域               0
市区町村名            0
地区名              0
最寄駅：距離（分）       23
取引価格（総額）         0
面積（u）            0
土地の形状            5
間口            1523
延床面積（u）        502
建築年            694
建物の構造          817
用途             822
今後の利用目的          0
前面道路：方位          0
前面道路：種類          0
前面道路：幅員（ｍ）       0
都市計画            65
建ぺい率（％）        103
容積率（％）         103
取引時点             0
dtype: int64

In [1202]:
#最寄駅からの時間の欠損値を平均値で埋める．
#文字型のデータを一括で30(分)に変換する．
df["最寄駅：距離（分）"].replace({"30分?60分":30, "1H?1H30":30, "1H30?2H":30, "2H?":30}, inplace = True)

#文字型を実数型に変換．
df['最寄駅：距離（分）'] = df['最寄駅：距離（分）'].astype('float64')

df.loc[:,"最寄駅：距離（分）"] = df["最寄駅：距離（分）"].fillna(df["最寄駅：距離（分）"].mean())

df.isnull().sum()

種類               0
地域               0
市区町村名            0
地区名              0
最寄駅：距離（分）        0
取引価格（総額）         0
面積（u）            0
土地の形状            5
間口            1523
延床面積（u）        502
建築年            694
建物の構造          817
用途             822
今後の利用目的          0
前面道路：方位          0
前面道路：種類          0
前面道路：幅員（ｍ）       0
都市計画            65
建ぺい率（％）        103
容積率（％）         103
取引時点             0
dtype: int64

In [1203]:
#土地の形状と用途，建物の構造，都市計画を最頻値で埋める．
df['土地の形状']=df['土地の形状'].fillna(df['土地の形状'].mode().iloc[0])
df['用途']=df['用途'].fillna(df['用途'].mode().iloc[0])
df['建物の構造']=df['建物の構造'].fillna(df['建物の構造'].mode().iloc[0])
df['都市計画']=df['都市計画'].fillna(df['都市計画'].mode().iloc[0])

#建ぺい率と容積率を平均値で埋める．
df.loc[:,"建ぺい率（％）"] = df["建ぺい率（％）"].fillna(df["建ぺい率（％）"].mean())
df.loc[:,"容積率（％）"] = df["容積率（％）"].fillna(df["容積率（％）"].mean())
df.isnull().sum()

種類               0
地域               0
市区町村名            0
地区名              0
最寄駅：距離（分）        0
取引価格（総額）         0
面積（u）            0
土地の形状            0
間口            1523
延床面積（u）        502
建築年            694
建物の構造            0
用途               0
今後の利用目的          0
前面道路：方位          0
前面道路：種類          0
前面道路：幅員（ｍ）       0
都市計画             0
建ぺい率（％）          0
容積率（％）           0
取引時点             0
dtype: int64

In [1204]:
#建築年と延床面積，取引総額の総額が欠損している行を削除する．

df = df.dropna(subset=['建築年', '取引価格（総額）', '延床面積（u）'])


#建築年が戦前の行を削除する．建築年と延床面積，取引総額の総額が欠損している行を削除する．
df = df[df['建築年'] != '戦前']

#建築年と取引時点を整数型に変換．
df['年号'] = df['建築年'].str[:2]



df['和暦_年'] = df['建築年'].str[2:].str.replace('年','').replace('','0').astype(int)
print(df['和暦_年'])
df.loc[df['年号']=='昭和','建築年'] = df['和暦_年'] + 1925
df.loc[df['年号']=='平成','建築年'] = df['和暦_年'] + 1988
df.loc[df['年号']=='令和','建築年'] = df['和暦_年'] + 2018

# 昭和元年 : 1926年, 平成元年: 1989, 令和元年: 2019
df['取引時点'] = df['取引時点'].str[:4].astype(int) + df['取引時点'].str[6].astype(int) / 4

0        44
1        37
2        60
3        47
5        61
         ..
16780    44
16782    18
16783    56
16784    57
16785     2
Name: 和暦_年, Length: 15923, dtype: int32


In [1205]:
df.isnull().sum()

種類               0
地域               0
市区町村名            0
地区名              0
最寄駅：距離（分）        0
取引価格（総額）         0
面積（u）            0
土地の形状            0
間口            1318
延床面積（u）          0
建築年              0
建物の構造            0
用途               0
今後の利用目的          0
前面道路：方位          0
前面道路：種類          0
前面道路：幅員（ｍ）       0
都市計画             0
建ぺい率（％）          0
容積率（％）           0
取引時点             0
年号               0
和暦_年             0
dtype: int64

In [1206]:
#50.0m以上の間口がある行と，面積が2000m²以上となっている行を削除
df = df[df['間口'] != '50.0m以上']
df = df[df['面積（u）'] != '2000u以上']#エラー表示から，m²がuであらわされていることが判明．
df = df[df['延床面積（u）'] != '2000u以上']


#平均値で欠損値を埋める．
df['間口'] = df['間口'].astype('float64')
df.loc[:,"間口"] = df["間口"].fillna(df["間口"].mean())



#面積（u）と延べ床面積と建築年を実数型にする．
df['面積（u）'] = df['面積（u）'].astype('float64')
df['延床面積（u）'] = df['延床面積（u）'].astype('float64')
df['建築年'] = df['建築年'].astype('float64')

In [1207]:
df.isnull().sum()

種類            0
地域            0
市区町村名         0
地区名           0
最寄駅：距離（分）     0
取引価格（総額）      0
面積（u）         0
土地の形状         0
間口            0
延床面積（u）       0
建築年           0
建物の構造         0
用途            0
今後の利用目的       0
前面道路：方位       0
前面道路：種類       0
前面道路：幅員（ｍ）    0
都市計画          0
建ぺい率（％）       0
容積率（％）        0
取引時点          0
年号            0
和暦_年          0
dtype: int64

In [1208]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15870 entries, 0 to 16785
Data columns (total 23 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   種類          15870 non-null  object 
 1   地域          15870 non-null  object 
 2   市区町村名       15870 non-null  object 
 3   地区名         15870 non-null  object 
 4   最寄駅：距離（分）   15870 non-null  float64
 5   取引価格（総額）    15870 non-null  int64  
 6   面積（u）       15870 non-null  float64
 7   土地の形状       15870 non-null  object 
 8   間口          15870 non-null  float64
 9   延床面積（u）     15870 non-null  float64
 10  建築年         15870 non-null  float64
 11  建物の構造       15870 non-null  object 
 12  用途          15870 non-null  object 
 13  今後の利用目的     15870 non-null  object 
 14  前面道路：方位     15870 non-null  object 
 15  前面道路：種類     15870 non-null  object 
 16  前面道路：幅員（ｍ）  15870 non-null  float64
 17  都市計画        15870 non-null  object 
 18  建ぺい率（％）     15870 non-null  float64
 19  容積率（％）      15870 non-nul

In [1209]:
#年号と和暦を削除(不要なので)．
df=df.drop('年号', axis=1)
df=df.drop('和暦_年', axis=1)
df.head()



,種類,地域,市区町村名,地区名,最寄駅：距離（分）,取引価格（総額）,面積（u）,土地の形状,間口,延床面積（u）,建築年,建物の構造,用途,今後の利用目的,前面道路：方位,前面道路：種類,前面道路：幅員（ｍ）,都市計画,建ぺい率（％）,容積率（％）,取引時点
0,宅地(土地と建物),商業地,千代田区,神田佐久間町,4.0,260000000,85.0,ほぼ長方形,7.0,470.0,1969.0,ＲＣ,事務所、作業場、倉庫、駐車場,住宅,北,区道,16.0,商業地域,80.0,600.0,2021.00
1,宅地(土地と建物),住宅地,千代田区,神田佐久間町,4.0,85000000,115.0,長方形,8.2,30.0,1962.0,木造,倉庫,住宅,北,私道,2.2,商業地域,80.0,500.0,2020.25
2,宅地(土地と建物),商業地,千代田区,神田須田町,2.0,230000000,70.0,ほぼ長方形,7.6,230.0,1985.0,鉄骨造,事務所,住宅,北西,区道,6.0,商業地域,80.0,600.0,2020.50
3,宅地(土地と建物),商業地,千代田区,神田須田町,1.0,240000000,70.0,ほぼ長方形,7.6,230.0,1972.0,鉄骨造,住宅、事務所、駐車場,住宅,北西,区道,6.0,商業地域,80.0,600.0,2020.50
5,宅地(土地と建物),商業地,千代田区,神田須田町,1.0,130000000,35.0,ほぼ長方形,4.0,165.0,1986.0,ＳＲＣ,住宅、店舗、その他,住宅,南西,区道,8.0,商業地域,80.0,600.0,2020.50


In [1210]:
#質的なデータ(カテゴリーデータ)をダミー変数(0または1)にする．
df = pd.get_dummies(data=df, columns=['地区名'])
df = pd.get_dummies(data=df, columns=['種類'])
df = pd.get_dummies(data=df, columns=['地域'])
df = pd.get_dummies(data=df, columns=['市区町村名'])
df = pd.get_dummies(data=df, columns=['土地の形状'])
df = pd.get_dummies(data=df, columns=['建物の構造'])
df = pd.get_dummies(data=df, columns=['用途'])
df = pd.get_dummies(data=df, columns=['今後の利用目的'])
df = pd.get_dummies(data=df, columns=['前面道路：方位'])
df = pd.get_dummies(data=df, columns=['前面道路：種類'])
df = pd.get_dummies(data=df, columns=['都市計画'])



In [1211]:
#多重共線性の判定
threshold = 0.8 #ここは恣意性がある．
corr_matrix = df.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if column != 'Tip' if any(upper[column] > threshold)]
print('削除する変数：', to_drop)
print('削除する変数の数：', len(to_drop))


削除する変数： ['容積率（％）', '地域_商業地', '市区町村名_三宅村', '市区町村名_八丈町', '市区町村名_神津島村', '市区町村名_西多摩郡日の出町', '用途_駐車場', '前面道路：種類_接面道路無し故に種類無し', '都市計画_都市計画区域外']
削除する変数の数： 9


In [1212]:
#多重共線性の判定
df = df.drop(columns = to_drop)
df.head()


#前処理完了

,最寄駅：距離（分）,取引価格（総額）,面積（u）,間口,延床面積（u）,建築年,前面道路：幅員（ｍ）,建ぺい率（％）,取引時点,地区名_お花茶屋,地区名_たかの台,地区名_つくし野,地区名_ひばりが丘,地区名_ひばりが丘北,地区名_みつい台,地区名_みなみ野,地区名_むさし野,地区名_めじろ台,地区名_一ツ家,地区名_一ノ宮,地区名_一之江,地区名_一番町,地区名_七国,地区名_万町,地区名_万願寺,地区名_三ツ木,地区名_三ツ藤,地区名_三ノ輪,地区名_三内,地区名_三原台,地区名_三園,地区名_三好,地区名_三宿,地区名_三根,地区名_三沢,地区名_三田,地区名_三軒茶屋,地区名_三輪町,地区名_三輪緑山,地区名_上の原,地区名_上一色,地区名_上中里,地区名_上井草,地区名_上代継,地区名_上北台,地区名_上北沢,地区名_上十条,地区名_上原,地区名_上壱分方町,地区名_上大崎,地区名_上小山田町,地区名_上川原町,地区名_上川町,地区名_上恩方町,地区名_上板橋,地区名_上柚木,地区名_上水南町,地区名_上水新町,地区名_上水本町,地区名_上池台,地区名_上池袋,地区名_上清戸,地区名_上用賀,地区名_上目黒,地区名_上石原,地区名_上石神井,地区名_上石神井南町,地区名_上砂町,地区名_上祖師谷,地区名_上篠崎,地区名_上荻,地区名_上落合,地区名_上連雀,地区名_上野桜木,地区名_上野毛,地区名_上野町,地区名_上馬,地区名_上高井戸,地区名_上高田,地区名_上鷺宮,地区名_下丸子,地区名_下井草,地区名_下保谷,地区名_下宿,地区名_下小山田町,地区名_下恩方町,地区名_下柚木,地区名_下清戸,地区名_下目黒,地区名_下石原,地区名_下石神井,地区名_下篠崎町,地区名_下落合,地区名_下谷,地区名_下連雀,地区名_下里,地区名_下馬,地区名_下高井戸,地区名_世田谷,地区名_両国,地区名_並木町,地区名_中,地区名_中丸町,地区名_中井,地区名_中十条,地区名_中原,地区名_中台,地区名_中和泉,地区名_中央,地区名_中央本町,地区名_中央町,地区名_中山,地区名_中島町,地区名_中川,地区名_中延,地区名_中村,地区名_中村北,地区名_中村南,地区名_中板橋,地区名_中根,地区名_中沢,地区名_中清戸,地区名_中町,地区名_中目黒,地区名_中神町,地区名_中落合,地区名_中葛西,地区名_中藤,地区名_中里,地区名_中野,地区名_中野上町,地区名_中野山王,地区名_中野町,地区名_中馬込,地区名_丸山,地区名_丸山町,地区名_久が原,地区名_久保山町,地区名_久我山,地区名_久米川町,地区名_乞田,地区名_亀戸,地区名_亀有,地区名_亀沢,地区名_二俣尾,地区名_二宮,地区名_二宮東,地区名_二葉,地区名_五ノ神,地区名_五日市,地区名_五本木,地区名_井の頭,地区名_井口,地区名_井草,地区名_京島,地区名_今井,地区名_今寺,地区名_今川,地区名_今戸,地区名_仙川町,地区名_代々木,地区名_代沢,地区名_代田,地区名_仲六郷,地区名_仲原,地区名_仲宿,地区名_仲池上,地区名_仲町,地区名_伊奈,地区名_伊奈平,地区名_伊興,地区名_伊興本町,地区名_佃,地区名_住吉,地区名_住吉町,地区名_佐野,地区名_佐須町,地区名_余丁町,地区名_保塚町,地区名_保木間,地区名_保谷町,地区名_元代々木町,地区名_元八王子町,地区名_元和泉,地区名_元本郷町,地区名_元横山町,地区名_元浅草,地区名_元町,地区名_光町,地区名_入谷,地区名_入間町,地区名_八丁堀,地区名_八幡山,地区名_八幡町,地区名_八広,地区名_八木町,地区名_八雲,地区名_八雲台,地区名_六月,地区名_六木,地区名_六町,地区名_兵衛,地区名_内藤,地区名_円山町,地区名_冬木,地区名_分梅町,地区名_初台,地区名_初沢町,地区名_別所,地区名_前原町,地区名_前沢,地区名_前野町,地区名_加住町,地区名_加平,地区名_加美平,地区名_加賀,地区名_勝島,地区名_勝沼,地区名_北,地区名_北加平町,地区名_北千束,地区名_北原町,地区名_北品川,地区名_北大塚,地区名_北小岩,地区名_北山町,地区名_北嶺町,地区名_北新宿,地区名_北沢,地区名_北烏山,地区名_北田園,地区名_北町,地区名_北砂,地区名_北篠崎,地区名_北糀谷,地区名_北葛西,地区名_北野,地区名_北野台,地区名_北野町,地区名_北馬込,地区名_十条仲原,地区名_千ヶ瀬町,地区名_千人町,地区名_千住,地区名_千住中居町,地区名_千住仲町,地区名_千住元町,地区名_千住大川町,地区名_千住宮元町,地区名_千住寿町,地区名_千住旭町,地区名_千住東,地区名_千住桜木,地区名_千住橋戸町,地区名_千住河原町,地区名_千住緑町,地区名_千住関屋町,地区名_千住龍田町,地区名_千川,地区名_千早,地区名_千束,地区名_千歳,地区名_千歳台,地区名_千田,地区名_千石,地区名_千駄ケ谷,地区名_千駄木,地区名_千鳥,地区名_南,地区名_南つくし野,地区名_南久が原,地区名_南六郷,地区名_南千住,地区名_南千束,地区名_南台,地区名_南品川,地区名_南大井,地区名_南大塚,地区名_南大沢,地区名_南大泉,地区名_南大谷,地区名_南小岩,地区名_南常盤台,地区名_南平,地区名_南成瀬,地区名_南榎町,地区名_南水元,地区名_南池袋,地区名_南沢,地区名_南烏山,地区名_南田中,地区名_南田園,地区名_南町,地区名_南町田,地区名_南砂,地区名_南篠崎町,地区名_南花畑,地区名_南荻窪,地区名_南葛西,地区名_南蒲田,地区名_南街,地区名_南野,地区名_南長崎,地区名_南陽台,地区名_南雪谷,地区名_南青山,地区名_南馬込,地区名_南麻布,地区名_原,地区名_原小宮,地区名_原町,地区名_原町田,地区名_友田町,地区名_双葉町,地区名_古千谷本町,地区名_古石場,地区名_台東,地区名_台町,地区名_叶谷町,地区名_吉祥寺北町,地区名_吉祥寺南町,地区名_吉祥寺本町,地区名_吉祥寺東町,地区名_向丘,地区名_向原,地区名_向台町,地区名_向山,地区名_向島,地区名_向陽台,地区名_和泉,地区名_和泉本町,地区名_和田,地区名_和田町,地区名_善福寺,地区名_喜多見,地区名_喜平町,地区名_四つ木,地区名_四葉,地区名_四谷,地区名_四谷三栄町,地区名_四谷坂町,地区名_四谷本塩町,地区名_四谷町,地区名_回田町,地区名_図師町,地区名_国領町,地区名_土支田,地区名_坂下,地区名_坂浜,地区名_城山手,地区名_堀ノ内,地区名_堀之内,地区名_堀切,地区名_堀船,地区名_堤通,地区名_塩浜,地区名_境,地区名_境南町,地区名_墨田,地区名_多摩川,地区名_多摩平,地区名_多摩湖町,地区名_大丸,地区名_大久保,地区名_大井,地区名_大京町,地区名_大南,地区名_大原,地区名_大原町,地区名_大和田町,地区名_大和町,地区名_大坂上,地区名_大塚,地区名_大字二本木,地区名_大字大久野,地区名_大字宮,地区名_大字川辺堀之内,地区名_大字平井,地区名_大字新井,地区名_大字日野,地区名_大字殿ケ谷,地区名_大字熊川,地区名_大字石田,地区名_大字福生,地区名_大字福生二宮,地区名_大字箱根ケ崎,地区名_大字豊田,地区名_大字長岡長谷部,地区名_大字駒形富士山,地区名_大字高根,地区名_大宮,地区名_大山東町,地区名_大山町,地区名_大山西町,地区名_大山金井町,地区名_大岡山,地区名_大島,地区名_大崎,地区名_大杉,地区名_大森中,地区名_大森北,地区名_大森南,地区名_大森本町,地区名_大森東,地区名_大森西,地区名_大楽寺町,地区名_大横町,地区名_大橋,地区名_大沢,地区名_大沼町,地区名_大泉学園町,地区名_大泉町,地区名_大神町,地区名_大船町,地区名_大蔵,地区名_大蔵町,地区名_大谷口,地区名_大谷口上町,地区名_大谷口北町,地区名_大谷田,地区名_大谷町,地区名_大賀郷,地区名_大門,地区名_大門町,地区名_天ヶ瀬町,地区名_天沼,地区名_天神町,地区名_太子堂,地区名_奈良橋,地区名_奥戸,地区名_奥沢,地区名_子安町,地区名_学園,地区名_学園東町,地区名_学園町,地区名_学園西町,地区名_宇喜田町,地区名_宇奈根,地区名_宇津木町,地区名_宝町,地区名_宮下町,地区名_宮前,地区名_宮坂,地区名_宮城,地区名_宮本町,地区名_宮沢町,地区名_宮町,地区名_宮西町,地区名_富ケ谷,地区名_富久町,地区名_富士本,地区名_富士町,地区名_富士見台,地区名_富士見平,地区名_富士見町,地区名_富岡,地区名_寺田町,地区名_寿町,地区名_小丹波,地区名_小作台,地区名_小台,地区名_小和田,地区名_小宮町,地区名_小山,地区名_小山ヶ丘,地区名_小山台,地区名_小山田桜台,地区名_小山町,地区名_小島町,地区名_小川,地区名_小川東,地区名_小川東町,地区名_小川町,地区名_小川西町,地区名_小日向,地区名_小松川,地区名_小柳町,地区名_小比企町,地区名_小石川,地区名_小竹町,地区名_小茂根,地区名_小菅,地区名_小豆沢,地区名_小野路町,地区名_小門町,地区名_尾山台,地区名_山吹町,地区名_山崎町,地区名_山王,地区名_山田,地区名_山田町,地区名_岡本,地区名_岡田,地区名_岩戸北,地区名_岩戸南,地区名_岩淵町,地区名_岸,地区名_岸町,地区名_島根,地区名_川井,地区名_川口町,地区名

In [1213]:
from sklearn.model_selection import train_test_split
# plt.scatter('取引価格（総額）', '建築年', data=df)
# データセットを説明変数と目的変数に分割
X, y = df.drop('取引価格（総額）', axis=1), df['取引価格（総額）']
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
# X_eval, X_test, y_eval, y_test = train_test_split(X_val, y_val, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=42)

In [1214]:
!pip install lightgbm

In [1215]:
import lightgbm as lgb

from sklearn.model_selection import KFold

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import numpy as np
!pip install --upgrade pandas


# 5つのモデルを保存するリストの初期化
models = []

# 学習用データでの予測値を保存するデータフレームの初期化
df_train_preds = pd.DataFrame({'y_train': y_train})

# 検証用データでの予測値を保存するデータフレームの初期化
df_eval_preds = pd.DataFrame({'y_eval': [],
                              'y_eval_pred': []})

# テストデータでの予測値を保存するデータフレームの初期化
df_test_preds = pd.DataFrame({'y_test': y_test})
# インデックスが0からの連番になるように初期化

df_test_preds.reset_index(inplace=True, drop=True)

# R^2を保存するデータフレームの初期化
df_R2 = pd.DataFrame({'train': [],
                      'eval': [],
                      'test': []})

# RMSEを保存するデータフレームの初期化
df_RMSE =pd.DataFrame({'train': [],
                       'eval': [],
                       'test': []})


# ラウンド数の初期化
round_no = 0


# 【ブロック２：　モデルの学習】
# 学習データの数だけの数列（0行から最終行まで連番）
row_no_list = list(range(len(y_train)))

# KFoldクラスをインスタンス化（これを使って5分割する）
K_fold = KFold(n_splits=10, shuffle=True,  random_state=42)

# KFoldクラスで分割した回数だけ実行（ここでは5回）
for train_cv_no, eval_cv_no in K_fold.split(row_no_list, y_train):
    # ilocで取り出す行を指定
    X_train_cv = X_train.iloc[train_cv_no, :]
    y_train_cv = pd.Series(y_train).iloc[train_cv_no]
    X_eval_cv = X_train.iloc[eval_cv_no, :]
    y_eval_cv = pd.Series(y_train).iloc[eval_cv_no]
    
    # 学習用
    lgb_train = lgb.Dataset(X_train_cv, y_train_cv,
                            free_raw_data=False)
    # 検証用
    lgb_eval = lgb.Dataset(X_eval_cv, y_eval_cv, reference=lgb_train,
                           free_raw_data=False)
    
    # パラメータを設定
    params = {
#         'task': 'train',                # 学習、トレーニング ⇔　予測predict
#               'boosting_type': 'gbdt',        # 勾配ブースティング
              'objective': 'regression',      # 目的関数：回帰
              'metric': 'rmse',               # 分類モデルの性能を測る指標
              'learning_rate': 0.03,            # 学習率（初期値0.1）
#             learning_rate= 0.03,
#             'max_depth': 3,
#             'num_leaves': 10,
#             'feature_fraction':0.7,
#             'bagging_fraction':1,
#             'bagging_freq':20,
#             'n_jobs':5,        
#               'lambda_l1': 2, 
#               'max_depth' : 7
    }
    
    # 学習
    evaluation_results = {}                              # 学習の経過を保存する箱
    model = lgb.train(params,                            # 上記で設定したパラメータ
                      lgb_train,                         # 使用するデータセット
                      num_boost_round=10000000,              # 学習の回数
                      valid_names=['train', 'valid'],    # 学習経過で表示する名称
                      valid_sets=[lgb_train, lgb_eval],  # モデル検証のデータセット
                      evals_result=evaluation_results,   # 学習の経過を保存
                      early_stopping_rounds=20,          # アーリーストッピング
                      verbose_eval=0)                    # 学習の経過の非表示
    
    # 学習が終わったモデルをリストに保存
    models.append(model) 
    
    
    # 【ブロック３：　モデルで予測】
    # 学習したモデルで予測
    y_train_pred = model.predict(X_train_cv, num_iteration=model.best_iteration)
    y_eval_pred = model.predict(X_eval_cv, num_iteration=model.best_iteration)
    y_test_pred = model.predict(X_test, num_iteration=model.best_iteration)
    
    # 学習用データでの予測値をデータフレームに保存
    df_train_cv_pred = pd.DataFrame({round_no: y_train_pred},
                                     index=train_cv_no)
    df_train_preds = df_train_preds.join(df_train_cv_pred, how='left')
    
    # 検証用データでの予測値をデータフレームに保存
    df_eval_pred = pd.DataFrame({'y_eval': y_eval_cv,
                                 'y_eval_pred': y_eval_pred})
    df_eval_preds = df_eval_preds.append(df_eval_pred)    
    
    # テストデータでの予測値をデータフレームに保存
    df_test_cv_pred = pd.DataFrame({round_no: y_test_pred})
    df_test_preds = pd.concat([df_test_preds, df_test_cv_pred], axis=1)

    #  r2_score を計算
    train_score = r2_score(y_train_cv, y_train_pred)
    eval_score = r2_score(y_eval_cv, y_eval_pred)
    test_score = r2_score(y_test, y_test_pred)
    
    #  RMSE を計算
    train_RMSE_score = np.sqrt(mean_squared_error(y_train_cv, y_train_pred))
    eval_RMSE_score = np.sqrt(mean_squared_error(y_eval_cv, y_eval_pred))
    test_RMSE_score = np.sqrt(mean_squared_error(y_test, y_test_pred))
    
    # スコアを表示
    print('R^2 train: %.5f, eval: %.5f, test: %.5f' 
          % (train_score, eval_score, test_score),
         '  |  RMSE train: %.5f, eval: %.5f, test: %.5f' 
          % (train_RMSE_score, eval_RMSE_score, test_RMSE_score))
    
    # R2の保存
    df_R2_cv = pd.DataFrame({'train': [train_score],
                             'eval': [eval_score],
                             'test': [test_score]},
                             index=[round_no])
    df_R2 = df_R2.append(df_R2_cv)
    
    # RMSEの保存
    df_RMSE_cv =pd.DataFrame({'train': [train_RMSE_score],
                              'eval': [eval_RMSE_score],
                              'test': [test_RMSE_score]},
                              index=[round_no])
    df_RMSE = df_RMSE.append(df_RMSE_cv)
    
    # ラウンド数のカウンタを更新
    round_no += 1


# 保存したR2の平均値   
R2_ave = np.array(df_R2.mean()) #.to_numpy()

# 保存したRMSEの平均値   
RMSE_ave = np.array(df_RMSE.mean()) #.to_numpy()

# 平均値を表示
print('Average:')
print('R^2 train: %.5f, eval: %.5f, test: %.5f'
      % (R2_ave[0], R2_ave[1], R2_ave[2]),
      '  |  RMSE train: %.5f, eval: %.5f, test: %.5f'
      % (RMSE_ave[0], RMSE_ave[1], RMSE_ave[2]))

#Average:
# R^2 train: 0.90656, eval: 0.80768, test: 0.81796

Requirement already up-to-date: pandas in c:\users\takeh\anaconda3\lib\site-packages (1.4.2)
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.012868 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1214
[LightGBM] [Info] Number of data points in the train set: 11426, number of used features: 264
[LightGBM] [Info] Start training from score 66776877.297392
R^2 train: 0.88457, eval: 0.79733, test: 0.80784   |  RMSE train: 34847091.39359, eval: 37721772.80683, test: 39136663.02831
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002160 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1207
[LightGBM] [Info] Number of data points in the train set: 11426, number of used features: 260
[LightGB